## Task 1. Генерация файла 
Нужно сгенерировать файл, содержащий 5000 32-битных случайных целых чисел, каждое число на отдельной строке. содержащий 5000 32-битных случайных целых чисел, каждое число на отдельной строке. 

In [35]:
import sys
sys.path.insert(0, "/home/dmitry/Desktop/BigData/")

import random
from multiprocessing import Pool, Manager

import ray
import dask.bag as db
from pyspark import SparkContext, SparkConf
from src.utils.timer import timer

In [36]:
file_name = "random_numbers32.txt"

with open(file_name, "w") as f:
    for i in range(5000):
        random_number = random.randint(0, 2**32 - 1)
        f.write(str(random_number) + "\n")

## Task 2. Посчитать, какое суммарное количество простых множителей присутствует при факторизации всех чисел.

In [37]:
def factorize(n):
    factors = []
    i = 2
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

## Task 3. Реализовать подсчет 
    - простым последовательным алгоритмом
    - многопоточно (на CPython - multiprocessing), с использованием примитивов синхронизации 
    - с помощью Ray/Dask/PySpark
    - (*) с использованием Go (go-рутин) / корутин на Kotlin

### Простой последовательный алгоритм

In [38]:
@timer
def simple_count(file_name: str) -> int:
    total_prime_count = 0
    with open(file_name, "r") as f:
        for line in f:
            n = int(line.strip())
            prime_count = len(factorize(n))
            total_prime_count += prime_count

    return total_prime_count

### Многопоточно (на CPython - multiprocessing), с использованием примитивов синхронизации 

In [39]:
def worker(numbers, results):
    prime_count = 0
    for n in numbers:
        prime_count += len(factorize(n))
    results.append(prime_count)

@timer
def multiprocessing_count(file_name):
    # читаем числа из файла и делим их на части для каждого процесса
    with open(file_name, "r") as f:
        numbers = [int(line.strip()) for line in f]
    chunk_size = len(numbers) // 4
    chunks = [numbers[i:i+chunk_size] for i in range(0, len(numbers), chunk_size)]

    # создаем пул из 4 процессов и результаты, которые будут возвращаться процессами
    with Manager() as manager:
        results = manager.list()
        pool = Pool(processes=4)

        # запускаем каждый процесс на своей части данных
        jobs = [pool.apply_async(worker, (chunk, results)) for chunk in chunks]

        # ждем, пока все процессы закончат работу
        for job in jobs:
            job.get()

        total_prime_count = sum(results)

    return total_prime_count


### C помощью Ray

In [40]:
ray.init()

2023-05-14 15:52:47,405	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.10.6
Ray version:,2.4.0


In [41]:
@ray.remote
def count_primes(n):
    prime_count = len(factorize(n))
    return prime_count

@timer
def ray_count(file_name: str) -> int:
    total_prime_count = 0
    futures = []

    with open(file_name, "r") as f:
        for line in f:
            n = int(line.strip())
            future = count_primes.remote(n)
            futures.append(future)

    for result in ray.get(futures):
        total_prime_count += result

    return total_prime_count

### C помощью Dask

In [42]:
@timer
def dask_count(file_name: str) -> int:
    b = db.read_text(file_name)
    primes = b.map(lambda x: len(factorize(int(x.strip()))))
    total_prime_count = primes.fold(lambda x, y: x + y).compute()

    return total_prime_count

### C помощью PySpark

In [43]:
@timer
def pyspark_count(file_name: str) -> int:
    conf = SparkConf().setAppName("Simple Count")
    sc = SparkContext.getOrCreate(conf=conf)

    lines = sc.textFile(file_name)
    primes = lines.map(lambda x: len(factorize(int(x.strip()))))

    total_prime_count = primes.reduce(lambda x, y: x + y)

    return total_prime_count

## Task 4. Измерить время выполнения для каждого случая.

In [44]:
simple_count(file_name=file_name)

simple_count took 4.5509 secs


20678

In [51]:
multiprocessing_count(file_name)

multiprocessing_count took 2.0010 secs


20678

In [47]:
print(ray_count(file_name))
ray.shutdown()

ray_count took 3.2346 secs
20678


In [48]:
dask_count(file_name)

dask_count took 4.6905 secs


20678

In [49]:
pyspark_count(file_name)

pyspark_count took 3.0856 secs


20678

### Расчет с использованием Go рутин приведен в файле gorutine.go
Время выполнения 0.306 seconds, что быстрее любого из полученных решений на python